# Propose the best area for a new Vegan Restaurant in Toronto - Capstone Project Part 2

# Import needed libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import math
import requests
from sklearn.cluster import KMeans
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


# Build Demographics Data for Toronto neighbourhoods

Get the demographics data (2011) from wikipedia into a pandas dataframe

In [2]:
link = "https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods"
df = pd.read_html(link, header = 0)[1]

In [3]:
# Clean up the data and dropping unwanted columns

df = df[df.Name != 'Toronto CMA Average']
df = df.drop(columns = ['Census Tracts', '% Change in Population since 2001', 
                          'Second most common language (after English) by name',
                          'Second most common language (after English) by percentage',
                          'Map']
              )
# change Population variable to numeric
df['Population'] = df['Population'].apply(pd.to_numeric)

# rename the first column
df = df.rename(columns = {'Name' : 'Neighborhood'})

print(df.shape)
df.head()

(174, 8)


,Neighborhood,FM,Population,Land area (km2),Density (people/km2),Average Income,Transit Commuting %,% Renters
1,Agincourt,S,44577,12.45,3580,25750,11.1,5.9
2,Alderwood,E,11656,4.94,2360,35239,8.8,8.5
3,Alexandra Park,OCoT,4355,0.32,13609,19687,13.8,28.0
4,Allenby,OCoT,2513,0.58,4333,245592,5.2,3.4
5,Amesbury,NY,17318,3.51,4934,27546,16.4,19.7


In [4]:
# Define a function that uses geopy library to get the latitude and longitude values of a address.

def get_coordinates(address):
    try:
        geolocator = Nominatim(user_agent = "to_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        return [latitude, longitude]
    except:
        return [None, None]

In [14]:
# Check the function for Toronto
address = 'Toronto, Ontario'
lat = get_coordinates(address)[0]
log = get_coordinates(address)[1]
print(lat)
print(log)

43.6534817
-79.3839347


In [ ]:
# Create coordinate variables as empty
df['Latitude']  = ''
df['Longitude'] = ''

In [11]:
# Use the get_coordinates function to obtain the coordinates for each neighbourhood

for hood in df['Neighborhood']:
    if (df.loc[df['Neighborhood'] == hood,'Latitude'].isna().bool() or df.loc[df['Neighborhood'] == hood,'Longitude'].isna().bool() ):
        address = hood + ', Toronto, Ontario'
        df.loc[df['Neighborhood'] == hood,'Latitude']   = get_coordinates(address)[0]
        df.loc[df['Neighborhood'] == hood,'Longitude']  = get_coordinates(address)[1]

In [22]:
# Ckeck the data
df.head(10)
#df.tail(10)

,Neighborhood,FM,Population,Land area (km2),Density (people/km2),Average Income,Transit Commuting %,% Renters,Latitude,Longitude
0,Agincourt,S,44577,12.45,3580,25750,11.1,5.9,43.785353,-79.278549
1,Alderwood,E,11656,4.94,2360,35239,8.8,8.5,43.601717,-79.545232
2,Alexandra Park,OCoT,4355,0.32,13609,19687,13.8,28.0,43.650758,-79.404308
3,Allenby,OCoT,2513,0.58,4333,245592,5.2,3.4,43.711351,-79.553424
4,Amesbury,NY,17318,3.51,4934,27546,16.4,19.7,43.706162,-79.483492
5,Armour Heights,NY,4384,2.29,1914,116651,10.8,16.1,43.743944,-79.430851
6,Banbury,NY,6641,2.72,2442,92319,6.1,4.8,43.742796,-79.369957
7,Bathurst Manor,NY,14945,4.69,3187,34169,13.4,18.6,43.763893,-79.456367
8,Bay Street Corridor,OCoT,4787,0.11,43518,40598,17.1,49.3,43.664457,-79.387189
9,Bayview Village,NY,12280,4.14,2966,46752,14.4,15.6,43.769197,-79.376662


In [23]:
# Additional check on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Neighborhood          174 non-null    object 
 1   FM                    174 non-null    object 
 2   Population            174 non-null    int64  
 3   Land area (km2)       174 non-null    float64
 4   Density (people/km2)  174 non-null    int64  
 5   Average Income        174 non-null    int64  
 6   Transit Commuting %   174 non-null    float64
 7   % Renters             174 non-null    float64
 8   Latitude              174 non-null    float64
 9   Longitude             174 non-null    float64
dtypes: float64(5), int64(3), object(2)
memory usage: 13.7+ KB


In [21]:
# Save data for later use as geolocator may not respond
# df.to_csv('hood_final.csv')

In [18]:
# Obtain the coordinates for each neighbourhood from the saved file if geolocator may not respond
df = pd.read_csv('hood_final.csv')
df.head()

,Neighborhood,FM,Population,Land area (km2),Density (people/km2),Average Income,Transit Commuting %,% Renters,Latitude,Longitude
0,Agincourt,S,44577,12.45,3580,25750,11.1,5.9,43.785353,-79.278549
1,Alderwood,E,11656,4.94,2360,35239,8.8,8.5,43.601717,-79.545232
2,Alexandra Park,OCoT,4355,0.32,13609,19687,13.8,28.0,43.650758,-79.404308
3,Allenby,OCoT,2513,0.58,4333,245592,5.2,3.4,43.711351,-79.553424
4,Amesbury,NY,17318,3.51,4934,27546,16.4,19.7,43.706162,-79.483492


Create map of Toronto using latitude and longitude values

In [125]:
address = 'Toronto, Ontario, Canada'
latitude = get_coordinates(address)[0]
longitude = get_coordinates(address)[1]

map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 11)

# add markers to map
for lat, lng, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Neighborhood']):

    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)

    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)  

map_toronto

# Use Foursquare Data to Explore Neighborhoods in Toronto

In [25]:
# Define Foursquare Credentials and Version

CLIENT_ID = '3LE3KJSML3TH3INQ2QVLVAAFI35D0IPPRSFQUIZ0ZJHBL0KQ' # your Foursquare ID
CLIENT_SECRET = 'BQXHQH4VRAFABQIYZODKMZPBEEJEQ1ZILLVVORWRIBAH3X1G' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3LE3KJSML3TH3INQ2QVLVAAFI35D0IPPRSFQUIZ0ZJHBL0KQ
CLIENT_SECRET:BQXHQH4VRAFABQIYZODKMZPBEEJEQ1ZILLVVORWRIBAH3X1G


In [26]:
# Use the function from the lab to get venues
LIMIT  = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [27]:

# Run the function on each neighborhood and create a new dataframe called toronto_venues

toronto_venues = getNearbyVenues(names = df['Neighborhood'], 
                                 latitudes  = df['Latitude'], longitudes = df['Longitude']
                                 
                                 )

Agincourt
Alderwood
Alexandra Park
Allenby
Amesbury
Armour Heights
Banbury
Bathurst Manor
Bay Street Corridor
Bayview Village
Bayview Woods – Steeles
Bedford Park
Bendale
Birch Cliff
Bloor West Village
Bracondale Hill
Branson
Bridle Path
Brockton
Cabbagetown
Caribou Park
Carleton Village
Casa Loma
Centennial
Chaplin Estates
Christie Pits
Church and Wellesley
Clairville
Clairlea
Cliffcrest
Cliffside
Corktown
Crescent Town
Cricket Club
Davenport
Davisville
Deer Park
Discovery District
Distillery District/West Don Lands
Don Mills
Don Valley Village
Dorset Park
Dovercourt Park
Downsview
Dufferin Grove
Earlscourt
East Danforth
Eatonville
Eglinton East
Elia (Jane and Finch)
Eringate
Fairbank
Fashion District
Financial District
Flemingdon Park
Forest Hill
Fort York/Liberty Village
Garden District
Glen Park
Governor's Bridge/Bennington Heights
Grange Park
Graydon Hall
Guildwood
Harbord Village
Harbourfront / CityPlace
Harwood
Henry Farm
High Park North
Highland Creek
Hillcrest
Hoggs Hollow
Hum

In [28]:
# Save data for later use if foursquare may not respond
# df.to_csv('toronto_venues.csv')

In [29]:
# Check the output
print(toronto_venues.shape)
toronto_venues.head()

(3292, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,43.788009,-79.283882,Tim Hortons,43.785637,-79.279215,Coffee Shop
1,Alderwood,43.603541,-79.546409,Il Paesano Pizzeria & Restaurant,43.601280,-79.545028,Pizza Place
2,Alderwood,43.603541,-79.546409,Toronto Gymnastics International,43.599832,-79.542924,Gym
3,Alderwood,43.603541,-79.546409,Timothy's Pub,43.600165,-79.544699,Pub
4,Alderwood,43.603541,-79.546409,Pizza Pizza,43.605340,-79.547252,Pizza Place


In [52]:
# Extract the neighborthood and latitude + longitude for each vegan restaurant
col = ['Venue Category', 'Venue Latitude', 'Venue Longitude', 'Neighborhood']
df_vegan = toronto_venues[toronto_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant']
df_vegan = df_vegan[['Venue Category', 'Venue Latitude', 'Venue Longitude', 'Neighborhood']]
df_vegan.head()

,Venue Category,Venue Latitude,Venue Longitude,Neighborhood
30,Vegetarian / Vegan Restaurant,43.651904,-79.403312,Alexandra Park
65,Vegetarian / Vegan Restaurant,43.648048,-79.396008,Alexandra Park
75,Vegetarian / Vegan Restaurant,43.647944,-79.396121,Alexandra Park
266,Vegetarian / Vegan Restaurant,43.650412,-79.478477,Bloor West Village
446,Vegetarian / Vegan Restaurant,43.675053,-79.406715,Casa Loma


In [91]:
# Let's check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,2,2,2,2,2,2
Alderwood,9,9,9,9,9,9
Alexandra Park,100,100,100,100,100,100
Allenby,18,18,18,18,18,18
Amesbury,5,5,5,5,5,5
Armour Heights,1,1,1,1,1,1
Banbury,1,1,1,1,1,1
Bathurst Manor,4,4,4,4,4,4
Bay Street Corridor,83,83,83,83,83,83


In [30]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 311 uniques categories.


In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep = "")
toronto_onehot.drop(['Neighborhood'], axis = 1, inplace = True) 

# add neighborhood column back to dataframe
toronto_onehot.insert(loc = 0, column = 'Neighborhood', value = toronto_venues['Neighborhood'] )

print(toronto_onehot.shape)
toronto_onehot.head()

(3292, 311)


,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,College Lab,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Latin American Restaurant,Laundromat,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Supplement Shop,Sush

In [32]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

# Save data for later use if needed
# df.to_csv('toronto_grouped.csv')

print(toronto_grouped.shape)
toronto_grouped.head()

(172, 311)


,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,College Lab,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Latin American Restaurant,Laundromat,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Supplement Shop,Sush

# Let's put that into a pandas dataframe the top 10 venues for each neighborhood

In [33]:
# First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(172, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Coffee Shop,Yoga Studio,Falafel Restaurant,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
1,Alderwood,Pizza Place,Pub,Gym,Convenience Store,Pharmacy,Donut Shop,Coffee Shop,Skating Rink,Pool,Hong Kong Restaurant
2,Alexandra Park,Bar,Café,Caribbean Restaurant,Vegetarian / Vegan Restaurant,Furniture / Home Store,Coffee Shop,Sushi Restaurant,Spa,Dessert Shop,Liquor Store
3,Allenby,Sushi Restaurant,Coffee Shop,Italian Restaurant,Lingerie Store,Clothing Store,Café,Bank,Bakery,Bagel Shop,Liquor Store
4,Amesbury,Portuguese Restaurant,Park,Bakery,Fast Food Restaurant,Restaurant,Flea Market,Farmers Market,Field,Filipino Restaurant,Fish & Chips Shop


# Create a lifestyle database

In [35]:
# Find categories that may be related to a healty lifestyle. These categories will be kept in the final database.

list = ['health food', 'organic', 'fruits', 'vegetables', 'vegetarian', 'vegan', 'gym', 'fitness', 'sport', 'yoga']
for word in list:
    print(word)
    for column in toronto_grouped.columns:
        if word in column.lower():
            print(column)

health food
Health Food Store
organic
Organic Grocery
fruits
vegetables
vegetarian
Vegetarian / Vegan Restaurant
vegan
Vegetarian / Vegan Restaurant
gym
College Gym
Gym
Gym / Fitness Center
fitness
Gym / Fitness Center
sport
Athletics & Sports
Sporting Goods Shop
Sports Bar
Sports Club
yoga
Yoga Studio


In [36]:
# We will keep in life style database the following categories

list = ['Health Food Store', 'Organic Grocery', 'Vegetarian / Vegan Restaurant', 'Gym / Fitness Center', 'Gym',
       'Athletics & Sports', 'Sporting Goods Shop', 'Sports Club', 'Yoga Studio']

df_lifestyle = pd.DataFrame()
df_lifestyle['Neighborhood'] = toronto_grouped['Neighborhood']

for column in list:
    df_lifestyle[column] = toronto_grouped[column]

print(df_lifestyle.shape)
df_lifestyle.head()

(172, 10)


,Neighborhood,Health Food Store,Organic Grocery,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Athletics & Sports,Sporting Goods Shop,Sports Club,Yoga Studio
0,Agincourt,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.00
1,Alderwood,0.0,0.00,0.00,0.00,0.100000,0.0,0.0,0.0,0.00
2,Alexandra Park,0.0,0.01,0.03,0.01,0.020000,0.0,0.0,0.0,0.02
3,Allenby,0.0,0.00,0.00,0.00,0.045455,0.0,0.0,0.0,0.00
4,Amesbury,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.00


# Create a database with the number of vegan restaurants for each neighborhood

In [37]:
# Save a databse with neighborhoods and number of vegan restaurants

toronto_gr = toronto_onehot.groupby('Neighborhood').sum().reset_index()
df_vegan_restaurants = toronto_gr[['Neighborhood','Vegetarian / Vegan Restaurant']]

print(df_vegan_restaurants.shape)
print('The total number of vegan restaurants is {}'.format(df_vegan_restaurants['Vegetarian / Vegan Restaurant'].sum()))

df_vegan_restaurants.head()

(172, 2)
The total number of vegan restaurants is 25


,Neighborhood,Vegetarian / Vegan Restaurant
0,Agincourt,0
1,Alderwood,0
2,Alexandra Park,3
3,Allenby,0
4,Amesbury,0


# Merge top 10 venues and lifestyle databases

In [38]:
# The databases will be merge by neighborhood column
df_venues = pd.merge(neighborhoods_venues_sorted, df_lifestyle, on = 'Neighborhood')
print(df_venues.shape)
df_venues.head()

(172, 20)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Health Food Store,Organic Grocery,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Athletics & Sports,Sporting Goods Shop,Sports Club,Yoga Studio
0,Agincourt,Coffee Shop,Yoga Studio,Falafel Restaurant,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.00
1,Alderwood,Pizza Place,Pub,Gym,Convenience Store,Pharmacy,Donut Shop,Coffee Shop,Skating Rink,Pool,Hong Kong Restaurant,0.0,0.00,0.00,0.00,0.100000,0.0,0.0,0.0,0.00
2,Alexandra Park,Bar,Café,Caribbean Restaurant,Vegetarian / Vegan Restaurant,Furniture / Home Store,Coffee Shop,Sushi Restaurant,Spa,Dessert Shop,Liquor Store,0.0,0.01,0.03,0.01,0.020000,0.0,0.0,0.0,0.02
3,Allenby,Sushi Restaurant,Coffee Shop,Italian Restaurant,Lingerie Store,Clothing Store,Café,Bank,Bakery,Bagel Shop,Liquor Store,0.0,0.00,0.00,0.00,0.045455,0.0,0.0,0.0,0.00
4,Amesbury,Portuguese Restaurant,Park,Bakery,Fast Food Restaurant,Restaurant,Flea Market,Farmers Market,Field,Filipino Restaurant,Fish & Chips Shop,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.00


# Add demographic data

In [108]:
# Merge with demografic data
df_final = pd.merge(df1, df_venues, on = 'Neighborhood')
print(df_final.shape)
df_final.head()

(172, 29)


,Neighborhood,FM,Population,Land area (km2),Density (people/km2),Average Income,Transit Commuting %,% Renters,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Health Food Store,Organic Grocery,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Athletics & Sports,Sporting Goods Shop,Sports Club,Yoga Studio
0,Agincourt,S,44577,12.45,3580,25750,11.1,5.9,43.788009,-79.283882,Coffee Shop,Yoga Studio,Falafel Restaurant,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.00
1,Alderwood,E,11656,4.94,2360,35239,8.8,8.5,43.603541,-79.546409,Pizza Place,Pub,Gym,Convenience Store,Pharmacy,Donut Shop,Coffee Shop,Skating Rink,Pool,Hong Kong Restaurant,0.0,0.00,0.00,0.00,0.100000,0.0,0.0,0.0,0.00
2,Alexandra Park,OCoT,4355,0.32,13609,19687,13.8,28.0,43.649778,-79.401457,Bar,Café,Caribbean Restaurant,Vegetarian / Vegan Restaurant,Furniture / Home Store,Coffee Shop,Sushi Restaurant,Spa,Dessert Shop,Liquor Store,0.0,0.01,0.03,0.01,0.020000,0.0,0.0,0.0,0.02
3,Allenby,OCoT,2513,0.58,4333,245592,5.2,3.4,43.707746,-79.412714,Sushi Restaurant,Coffee Shop,Italian Restaurant,Lingerie Store,Clothing Store,Café,Bank,Bakery,Bagel Shop,Liquor Store,0.0,0.00,0.00,0.00,0.045455,0.0,0.0,0.0,0.00
4,Amesbury,NY,17318,3.51,4934,27546,16.4,19.7,43.701093,-79.481006,Portuguese Restaurant,Park,Bakery,Fast Food Restaurant,Restaurant,Flea Market,Farmers Market,Field,Filipino Restaurant,Fish & Chips Shop,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.00


In [109]:
# Final check on data
df_final.info()

# Save data as csv file
# df.to_csv('final.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 171
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Neighborhood                   172 non-null    object 
 1   FM                             172 non-null    object 
 2   Population                     172 non-null    int64  
 3   Land area (km2)                172 non-null    float64
 4   Density (people/km2)           172 non-null    int64  
 5   Average Income                 172 non-null    int64  
 6   Transit Commuting %            172 non-null    float64
 7   % Renters                      172 non-null    float64
 8   Latitude                       172 non-null    float64
 9   Longitude                      172 non-null    float64
 10  1st Most Common Venue          172 non-null    object 
 11  2nd Most Common Venue          172 non-null    object 
 12  3rd Most Common Venue          172 non-null    obj

# Cluster Neighborhoods

In [110]:
# Prepare data for cluster analysis by standardizing numerical variables in demographic data (df1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df2 = df1.drop(columns = ['Neighborhood', 'FM'], axis = 1)
columns = df2.columns
df3 = pd.DataFrame(scaler.fit_transform(df2), columns = columns)

df3.insert(0, column = 'Neighborhood', value = df1['Neighborhood']) 
df3.head()

#toronto_grouped_clustering.head()

,Neighborhood,Population,Land area (km2),Density (people/km2),Average Income,Transit Commuting %,% Renters,Latitude,Longitude
0,Agincourt,2.982961,2.521411,-0.392108,-0.563413,-0.919259,-1.218321,1.644795,1.155513
1,Alderwood,-0.216507,0.406250,-0.572779,-0.336724,-1.406656,-0.949642,-1.980548,-1.447805
2,Alexandra Park,-0.926064,-0.894954,1.093096,-0.708256,-0.347097,1.065451,-1.071856,-0.010400
3,Allenby,-1.105081,-0.821726,-0.280596,4.688543,-2.169538,-1.476666,0.067381,-0.122036
4,Amesbury,0.333761,0.003496,-0.191593,-0.520507,0.203874,0.207745,-0.063357,-0.799247


In [111]:
# Merge numerrical data with venues data

df4 = pd.merge(df3, toronto_grouped, on = 'Neighborhood')

toronto_grouped_clustering = df4.drop(columns = ['Neighborhood'], axis = 1)
toronto_grouped_clustering.head()

,Population,Land area (km2),Density (people/km2),Average Income,Transit Commuting %,% Renters,Latitude,Longitude,Accessories Store,Adult Boutique,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,College Lab,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Latin American Restaurant,Laundromat,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri L

In [112]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([3, 1, 0, 4, 1, 4, 4, 1, 2, 3, 3, 1, 3, 3, 0, 0, 0, 0, 0, 4, 0, 0,
       3, 0, 0, 2, 1, 0, 3, 3, 0, 0, 4, 0, 0, 0, 0, 0, 3, 3, 3, 0, 3, 0,
       1, 0, 1, 0, 3, 1, 0, 0, 0, 0, 1, 0, 0, 1, 4, 0, 0, 3, 0, 0, 1, 0,
       0, 3, 1, 4, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 3, 0, 1, 0,
       4, 4, 4, 0, 0, 0, 1, 4, 3, 1, 1, 3, 3, 1, 4, 3, 0, 1, 3, 0, 2, 0,
       3, 3, 4, 0, 0, 1, 0, 3, 1, 3, 1, 0, 0, 1, 0, 1, 0, 4, 3, 3, 1, 3,
       3, 3, 0, 1, 1, 0, 2, 3, 0, 1, 1, 3, 0, 0, 0, 1, 0, 4, 1, 1, 0, 1,
       0, 0, 3, 0, 1, 3, 3, 0, 1, 0, 3, 3, 1, 3, 0, 3, 3, 0])

In [113]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

toronto_merged = df_final 
toronto_merged.insert(0, 'Cluster Labels', kmeans.labels_.astype('int32'))

In [114]:
# Check the data
toronto_merged.head()

,Cluster Labels,Neighborhood,FM,Population,Land area (km2),Density (people/km2),Average Income,Transit Commuting %,% Renters,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Health Food Store,Organic Grocery,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Athletics & Sports,Sporting Goods Shop,Sports Club,Yoga Studio
0,3,Agincourt,S,44577,12.45,3580,25750,11.1,5.9,43.788009,-79.283882,Coffee Shop,Yoga Studio,Falafel Restaurant,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.00
1,1,Alderwood,E,11656,4.94,2360,35239,8.8,8.5,43.603541,-79.546409,Pizza Place,Pub,Gym,Convenience Store,Pharmacy,Donut Shop,Coffee Shop,Skating Rink,Pool,Hong Kong Restaurant,0.0,0.00,0.00,0.00,0.100000,0.0,0.0,0.0,0.00
2,0,Alexandra Park,OCoT,4355,0.32,13609,19687,13.8,28.0,43.649778,-79.401457,Bar,Café,Caribbean Restaurant,Vegetarian / Vegan Restaurant,Furniture / Home Store,Coffee Shop,Sushi Restaurant,Spa,Dessert Shop,Liquor Store,0.0,0.01,0.03,0.01,0.020000,0.0,0.0,0.0,0.02
3,4,Allenby,OCoT,2513,0.58,4333,245592,5.2,3.4,43.707746,-79.412714,Sushi Restaurant,Coffee Shop,Italian Restaurant,Lingerie Store,Clothing Store,Café,Bank,Bakery,Bagel Shop,Liquor Store,0.0,0.00,0.00,0.00,0.045455,0.0,0.0,0.0,0.00
4,1,Amesbury,NY,17318,3.51,4934,27546,16.4,19.7,43.701093,-79.481006,Portuguese Restaurant,Park,Bakery,Fast Food Restaurant,Restaurant,Flea Market,Farmers Market,Field,Filipino Restaurant,Fish & Chips Shop,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.00


# Create map with the neighborhood clusters and the vegan restaurands

In [115]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start = 10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup  = label,
        color  = rainbow[cluster-1],
        fill   = True,
        fill_color   = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)

# add vegan restaurant to the map (as brown points)
for lat, lon, nei,  in zip(df_vegan['Venue Latitude'], df_vegan['Venue Longitude'], df_vegan['Neighborhood']):
    label = folium.Popup(str(nei + 'Vegan'), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 1,
        popup  = label,
        color  = 'brown',
        fill   = False,
        fill_color   = 'brown',
        fill_opacity = 0.3).add_to(map_clusters)   
       
map_clusters

# Examine clusters

In [116]:
# Cluster 0 ==> Concentrated on center of Toronto, with lots of restaurants and shops and income over the average.
# Livestyle: lots of gym locations and healty food stores.
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0,:]

,Cluster Labels,Neighborhood,FM,Population,Land area (km2),Density (people/km2),Average Income,Transit Commuting %,% Renters,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Health Food Store,Organic Grocery,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Athletics & Sports,Sporting Goods Shop,Sports Club,Yoga Studio
2,0,Alexandra Park,OCoT,4355,0.32,13609,19687,13.8,28.0,43.649778,-79.401457,Bar,Café,Caribbean Restaurant,Vegetarian / Vegan Restaurant,Furniture / Home Store,Coffee Shop,Sushi Restaurant,Spa,Dessert Shop,Liquor Store,0.000000,0.010000,0.030000,0.010000,0.020000,0.000000,0.000000,0.0,0.020000
14,0,Bloor West Village,OCoT,5175,0.74,6993,55578,20.7,8.3,43.650863,-79.477651,Café,Coffee Shop,Bakery,Pizza Place,Falafel Restaurant,Bank,Burrito Place,Frozen Yogurt Shop,Tea Room,Gourmet Shop,0.000000,0.000000,0.022222,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
15,0,Bracondale Hill,OCoT,5343,0.62,8618,41605,18.2,24.2,43.678237,-79.426784,Italian Restaurant,Coffee Shop,Restaurant,Mexican Restaurant,Café,Bakery,Indian Restaurant,Event Space,Farmers Market,Fish & Chips Shop,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
16,0,Branson,NY,8017,1.25,6414,27156,18.0,27.2,43.776421,-79.450699,Skating Rink,Park,Locksmith,Yoga Studio,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
17,0,Brockton,OCoT,9039,1.10,8217,27260,22.3,17.3,43.653191,-79.438039,Coffee Shop,Grocery Store,Bar,Restaurant,Vietnamese Restaurant,Clothing Store,Bakery,Department Store,Fast Food Restaurant,Portuguese Restaurant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
18,0,Cabbagetown,OCoT,11120,1.40,7943,50398,18.5,29.6,43.667263,-79.369169,Coffee Shop,Restaurant,Pizza Place,Italian Restaurant,Pub,Diner,Café,Bakery,Gastropub,Japanese Restaurant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
20,0,Carleton Village,OCoT,6544,0.74,8843,23301,19.0,19.9,43.671630,-79.457731,Vietnamese Restaurant,Bus Line,Burger Joint,Fish Market,Pizza Place,Brazilian Restaurant,Jewelry Store,Bakery,Light Rail Station,Food & Drink Shop,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
21,0,Casa Loma,OCoT,3597,0.67,5369,82203,18.1,32.9,43.678037,-79.409444,Sandwich Place,History Museum,Café,Coffee Shop,Burger Joint,Donut Shop,Museum,Steakhouse,Middle Eastern Restaurant,Castle,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
23,0,Chaplin Estates,OCoT,4906,0.93,5275,81288,22.7,24.5,43.701954,-79.401457,Italian Restaurant,Sushi Restaurant,Burger Joint,Restaurant,Middle Eastern Restaurant,French Restaurant,Breakfast Spot,Gastropub,Syrian Restaurant,General Entertainment,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
24,0,Christie Pits,OCoT,5124,0.64,8006,30556,18.8,15.3,43.664589,-79.420681,Korean Restaurant,Café,Grocery Store,Cocktail Bar,Pub,Mexican Restaurant,Coffee Shop,Ethiopian Restaurant,Ice Cream Shop,Indian Restaurant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [117]:
# Cluster 1 ==> Covers a big area, low population density, lowest income and standard venues. 
# Lifestyle: interested in sport but not in healty food.
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, :]

,Cluster Labels,Neighborhood,FM,Population,Land area (km2),Density (people/km2),Average Income,Transit Commuting %,% Renters,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Health Food Store,Organic Grocery,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Athletics & Sports,Sporting Goods Shop,Sports Club,Yoga Studio
1,1,Alderwood,E,11656,4.94,2360,35239,8.8,8.5,43.603541,-79.546409,Pizza Place,Pub,Gym,Convenience Store,Pharmacy,Donut Shop,Coffee Shop,Skating Rink,Pool,Hong Kong Restaurant,0.0,0.0,0.0,0.000000,0.100000,0.000000,0.000000,0.0,0.0
4,1,Amesbury,NY,17318,3.51,4934,27546,16.4,19.7,43.701093,-79.481006,Portuguese Restaurant,Park,Bakery,Fast Food Restaurant,Restaurant,Flea Market,Farmers Market,Field,Filipino Restaurant,Fish & Chips Shop,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
7,1,Bathurst Manor,NY,14945,4.69,3187,34169,13.4,18.6,43.762740,-79.456325,Baseball Field,Park,Playground,Convenience Store,Flower Shop,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
11,1,Bedford Park,OCoT,13749,2.27,6057,80827,15.2,10.1,43.730125,-79.411307,Juice Bar,Construction & Landscaping,Pizza Place,Food,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
26,1,Clairville,E,8506,6.71,1268,26610,13.2,7.2,43.742504,-79.622137,Department Store,Music Venue,Food,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
44,1,Earlscourt,OCoT,17240,2.24,7696,26672,17.7,8.5,43.676906,-79.450699,Italian Restaurant,Bakery,Burger Joint,Breakfast Spot,Bank,Park,Coffee Shop,Sushi Restaurant,Brazilian Restaurant,Café,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
46,1,Eatonville,E,19131,11.26,1699,36206,12.6,13.4,43.641448,-79.554724,Lawyer,Yoga Studio,Falafel Restaurant,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
49,1,Eringate,E,8008,2.44,3282,34789,10.9,4.0,43.662504,-79.577201,Park,Chinese Restaurant,Convenience Store,Pizza Place,Eastern European Restaurant,Yoga Studio,Flea Market,Field,Filipino Restaurant,Fish & Chips Shop,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
54,1,Forest Hill,OCoT,24056,4.35,5530,101631,14.8,21.9,43.704318,-79.426784,Bagel Shop,Pizza Place,Coffee Shop,Dance Studio,Frozen Yogurt Shop,Middle Eastern Restaurant,Spa,Bank,Korean Restaurant,Grocery Store,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
57,1,Glen Park,NY,18426,4.13,4462,35022,13.6,17.0,43.710101,-79.450699,Furniture / Home Store,Gas Station,Latin American Restaurant,Mediterranean Restaurant,Bookstore,Pizza Place,Grocery Store,Italian Restaurant,Ice Cream Shop,Food Service,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [118]:
# Cluster 2 ==> Small areas close to center of Toronto, with highest population density and the lowest average income. 
# Lifestyle: low interest in sport and no interest in healthy food.
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, :]

,Cluster Labels,Neighborhood,FM,Population,Land area (km2),Density (people/km2),Average Income,Transit Commuting %,% Renters,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Health Food Store,Organic Grocery,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Athletics & Sports,Sporting Goods Shop,Sports Club,Yoga Studio
8,2,Bay Street Corridor,OCoT,4787,0.11,43518,40598,17.1,49.3,43.656731,-79.383460,Coffee Shop,Sandwich Place,Clothing Store,Hotel,Ramen Restaurant,Bubble Tea Shop,Fried Chicken Joint,Café,Bar,Middle Eastern Restaurant,0.0,0.0,0.0,0.012048,0.000000,0.0,0.0,0.0,0.000000
25,2,Church and Wellesley,OCoT,13397,0.55,24358,37653,25.1,57.0,43.664542,-79.381752,Japanese Restaurant,Burger Joint,Sushi Restaurant,Coffee Shop,Ramen Restaurant,Diner,Gastropub,Gay Bar,Dance Studio,Men's Store,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.014286
108,2,North York City Centre,NY,10427,0.28,37239,34330,19.0,19.5,43.768303,-79.413332,Pizza Place,Coffee Shop,Ramen Restaurant,Sandwich Place,Restaurant,Japanese Restaurant,Dessert Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,0.0,0.0,0.0,0.000000,0.018182,0.0,0.0,0.0,0.000000
138,2,St. James Town,OCoT,14666,0.23,63765,22341,27.4,46.2,43.670867,-79.373306,Coffee Shop,Pizza Place,Grocery Store,Metro Station,Convenience Store,Caribbean Restaurant,Food & Drink Shop,Breakfast Spot,Market,Filipino Restaurant,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000


In [119]:
# Cluster 3 ==> Big area with low density of population and lowest income. Sport venues are present but no interest in healthy food.
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, :]

,Cluster Labels,Neighborhood,FM,Population,Land area (km2),Density (people/km2),Average Income,Transit Commuting %,% Renters,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Health Food Store,Organic Grocery,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Athletics & Sports,Sporting Goods Shop,Sports Club,Yoga Studio
0,3,Agincourt,S,44577,12.45,3580,25750,11.1,5.9,43.788009,-79.283882,Coffee Shop,Yoga Studio,Falafel Restaurant,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,3,Bayview Village,NY,12280,4.14,2966,46752,14.4,15.6,43.778243,-79.382758,Trail,Gas Station,Yoga Studio,Flower Shop,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
10,3,Bayview Woods – Steeles,NY,13298,4.07,3267,41485,11.2,13.9,43.796411,-79.385975,Park,Dog Run,Yoga Studio,Flower Shop,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
12,3,Bendale,S,28945,8.49,3409,29723,13.0,10.4,43.765434,-79.248935,Trail,Home Service,Yoga Studio,Flea Market,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
13,3,Birch Cliff,S,12266,3.48,3525,48965,11.4,15.2,43.687500,-79.269444,Pub,Bar,Park,Diner,Thai Restaurant,Beach,Indian Restaurant,Arepa Restaurant,Farmers Market,Field,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
22,3,Centennial,E,12565,4.94,2544,34867,11.5,8.8,43.784161,-79.149355,Park,Flower Shop,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Yoga Studio,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
28,3,Cliffcrest,S,14531,7.01,2073,38182,12.5,12.2,43.724917,-79.226329,Pharmacy,Coffee Shop,Bistro,Sandwich Place,Yoga Studio,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
29,3,Cliffside,S,9386,2.45,3831,32701,14.8,11.4,43.712217,-79.244001,Sandwich Place,Breakfast Spot,Grocery Store,Pizza Place,Yoga Studio,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
38,3,Don Mills,NY,21372,8.99,2377,47515,10.8,18.6,43.744847,-79.340923,Japanese Restaurant,Café,Caribbean Restaurant,Yoga Studio,Food,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
39,3,Don Valley Village,NY,29740,5.16,5764,30442,17.1,16.6,43.783779,-79.352188,Baseball Field,Fruit & Vegetable Store,Gas Station,Beer Store,Bakery,Park,Sandwich Place,Grocery Store,Caribbean Restaurant,Pizza Place,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [120]:
# Cluster 4 ==> Selective area with lowest population density and the highest average income, much over the average. 
# Healthy food seems to be much below expectations. Lifestyle - green areas with many sport lovers. 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, :]

,Cluster Labels,Neighborhood,FM,Population,Land area (km2),Density (people/km2),Average Income,Transit Commuting %,% Renters,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Health Food Store,Organic Grocery,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Athletics & Sports,Sporting Goods Shop,Sports Club,Yoga Studio
3,4,Allenby,OCoT,2513,0.58,4333,245592,5.2,3.4,43.707746,-79.412714,Sushi Restaurant,Coffee Shop,Italian Restaurant,Lingerie Store,Clothing Store,Café,Bank,Bakery,Bagel Shop,Liquor Store,0.0,0.0,0.0,0.0,0.045455,0.000000,0.000000,0.0,0.0
5,4,Armour Heights,NY,4384,2.29,1914,116651,10.8,16.1,43.745388,-79.422564,Wine Shop,Yoga Studio,Flower Shop,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
6,4,Banbury,NY,6641,2.72,2442,92319,6.1,4.8,43.749115,-79.366359,Park,Soccer Field,Yoga Studio,Falafel Restaurant,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
19,4,Caribou Park,NY,4097,1.02,4017,103703,8.4,7.6,43.718276,-79.423971,Sushi Restaurant,Middle Eastern Restaurant,Bagel Shop,Market,Yoga Studio,Flower Shop,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
32,4,Cricket Club,NY,5504,1.96,2808,104362,8.7,15.4,43.738548,-79.412714,Sports Club,Yoga Studio,Food,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0
58,4,Governor's Bridge/Bennington Heights,EY,2112,1.87,1129,129904,7.1,13.3,43.693193,-79.372238,Park,Pharmacy,Grocery Store,Gas Station,Bank,Martial Arts Dojo,Kids Store,Yoga Studio,Field,Filipino Restaurant,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
69,4,Hoggs Hollow,NY,3123,2.76,1132,222560,4.3,2.9,43.741970,-79.398642,Park,Bus Stop,Speakeasy,Yoga Studio,Flower Shop,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
88,4,Lawrence Park,OCoT,6653,3.64,1828,214110,8.3,4.6,43.723282,-79.388790,Jewelry Store,Yoga Studio,Food,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
89,4,Leaside,EY,13876,2.81,4938,82670,9.7,10.5,43.708826,-79.373499,Bakery,Coffee Shop,Indian Restaurant,Thai Restaurant,Asian Restaurant,Sushi Restaurant,Discount Store,Bank,Wings Joint,Supermarket,0.0,0.0,0.0,0.0,0.000000,0.000000,0.027778,0.0,0.0
90,4,Ledbury Park,NY,12360,2.50,4944,83853,9.5,12.2,43.731176,-79.422564,Italian Restaurant,Coffee Shop,Sandwich Place,Sushi Restaurant,Grocery Store,Skating Rink,Park,Comfort Food Restaurant,Bagel Shop,Liquor Store,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [121]:
# Characteristics of the clusters - demographics & lifestyle
df_cluster = toronto_merged.groupby('Cluster Labels').mean()

In [124]:
df_cluster = df_cluster.drop(columns = ['Latitude', 'Longitude']) 
df_cluster.head()

,Population,Land area (km2),Density (people/km2),Average Income,Transit Commuting %,% Renters,Health Food Store,Organic Grocery,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Athletics & Sports,Sporting Goods Shop,Sports Club,Yoga Studio
Cluster Labels,,,,,,,,,,,,,,,
0,11283.086957,1.518986,8131.608696,42210.797101,19.210145,24.153623,0.00128,0.000558,0.010462,0.007490,0.013315,0.004290,0.001502,0.000000,0.007736
1,12090.454545,4.157955,3394.409091,39283.909091,12.750000,13.143182,0.00000,0.000000,0.000000,0.006944,0.020351,0.011111,0.006581,0.000000,0.000000
2,10819.250000,0.292500,42220.000000,33730.500000,22.150000,43.000000,0.00000,0.000000,0.000000,0.003012,0.004545,0.000000,0.000000,0.000000,0.003571
3,23530.450000,7.007500,3750.775000,33651.650000,13.665000,12.407500,0.00000,0.000000,0.000000,0.004795,0.001452,0.002083,0.001324,0.000000,0.050490
4,6179.533333,2.062000,3232.600000,140877.466667,9.560000,9.333333,0.00000,0.000000,0.000000,0.000000,0.019697,0.005556,0.001852,0.066667,0.000000


# Conclusion 

The neighborhoods of Toronto were analyzed considering demographic characteristics, the venue categories and the lifestyle. 

We have found that there are two clusters of neighborhoods which are appropriate for vegan restaurants. 

When we consider the existing competition and the average income level the conclusion is clear and we recommended two areas around the neighborhoods of cluster 4-orange in our analysis. 